In [56]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import ComplementNB
from sklearn.feature_extraction import text
import nltk
import re
import math
from nltk import pos_tag, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')
from sklearn.model_selection import KFold
from sklearn.metrics import balanced_accuracy_score, accuracy_score, precision_score, recall_score

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/christinefang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/christinefang/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [117]:
df = pd.read_csv('../../../sample_events.csv')
##satp = satp[["description", "relevant_event"]]
df = df[["description", "internal_conflict", "multiple_events", "naxalite_maoist"]]
##df = df.replace()
df = df.drop_duplicates(subset=["description"]).reset_index(drop=True)
df["internal_conflict"] = df["internal_conflict"].replace(r'[2-9]{1,4}', np.nan, regex=True).fillna(0)
df.index = range(len(df))
df.head(10)

,description,internal_conflict
0,Nagaland Post reports that a National Socialis...,1.0
1,"On February 22, Minister of State for Home, S....",0.0
2,Kanglaonline reports on June 21 that a militan...,1.0
3,Two security personnel were seriously injured ...,1.0
4,An unnamed senior State Home department offic...,0.0
5,The Union Ministry of Home Affairs (UMHA) has ...,1.0
6,A woman Communist Party of India-Maoist (CPI-M...,1.0
7,Militants lobbed a hand grenade targeting comp...,1.0
8,Students under Kangleipak Students' Associatio...,1.0
9,"On May 5, an IED blast was reported in front o...",1.0


# Cleaning and Tokenizing

In [118]:
data_stripped = []
data_tokenized_words = []

wl = WordNetLemmatizer()

for i in range(len(df)):
    # removes unnecessary characters
    row = re.sub('[^a-zA-Z0-9. ]', '', df['description'][i])
    row = row.lower()
    # tokenization
    row = row.split()
    # lemmatization and stop words removal
    row = [wl.lemmatize(word) for word in row if not word in set(stopwords.words('english'))]
    row2 = re.sub('[^a-zA-Z0-9 ]', '', ' '.join(row))
    row2 = row2.split(' ')
    row3 = ' '.join(row).split('.')
    
    
    data_stripped.append(' '.join(row))
    data_tokenized_words.append(row2)


df['stripped_description'] = data_stripped
df['word_tokenized_description'] = data_tokenized_words

In [113]:
# Define function for cross-validation
def cross_validation(folds, Y, accuracy_metric):
    # Split dataset into folds
    kf = KFold(n_splits=folds, shuffle=True)
    accuracy = []
    # SVM model
    nb = MultinomialNB(force_alpha = True)
    vectorizer = text.TfidfVectorizer(analyzer=lambda x: x,)
    X = vectorizer.fit_transform(df["word_tokenized_description"])

    # Cross-validation
    for train_idx, valid_idx in kf.split(X):
        X_train, X_valid = X[train_idx, :], X[valid_idx, :]
        Y_train, Y_valid = Y[train_idx], Y[valid_idx]
        nb.fit(X_train,Y_train)
        predictions = nb.predict(X_valid)
        accuracy.append(accuracy_metric(predictions, Y_valid))
    return np.mean(accuracy)

In [130]:
def accuracies(Y):
        accuracy_cv = pd.DataFrame(columns = ['folds', 'validation_acc', 'balanced_acc', 'precision', 'recall'])

        metrics = [accuracy_score, balanced_accuracy_score, precision_score, recall_score]

        for folds in range(2,6):
                accuracies = [folds]
                for i in range(len(metrics)):
                        mean_accuracy = cross_validation(folds, Y, metrics[i])
                        accuracies.append(mean_accuracy)
                accuracy_cv = pd.concat([pd.DataFrame([accuracies], columns=accuracy_cv.columns), accuracy_cv], ignore_index=True)

        accuracy_cv['folds'] = accuracy_cv['folds'].astype('int')
        accuracy_cv.set_index('folds')
        return accuracy_cv



In [131]:
accuracies(df["internal_conflict"])

/var/folders/ny/vbk2t6r52hd7yvymd0bn98ww0000gp/T/ipykernel_66232/2213890824.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  accuracy_cv = pd.concat([pd.DataFrame([accuracies], columns=accuracy_cv.columns), accuracy_cv], ignore_index=True)
/Users/christinefang/Desktop/URAP/conflict-india/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:2399: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/Users/christinefang/Desktop/URAP/conflict-india/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:2399: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


,folds,validation_acc,balanced_acc,precision,recall
0,5,0.726410,0.692883,0.994586,0.719367
1,4,0.714660,0.706680,0.992823,0.717387
2,3,0.714652,0.748618,1.000000,0.715370
3,2,0.714660,0.732432,1.000000,0.713957


In [132]:
accuracies(df["multiple_events"])

KeyError: 'multiple_events'

In [41]:
count = 0
for i in range(len(prd)):
    if prd[i] == validy[i]:
        count += 1
print(f"validation accuracy: {count}/{len(prd)} = {count/len(prd)}")

validation accuracy: 115/152 = 0.756578947368421


In [43]:
# K-fold  CV
from sklearn.model_selection import cross_val_score
# 10-fold CV
scores = cross_val_score(nb, trainx, trainy, scoring = 'balanced_accuracy' , cv = 10)

scores

array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5])

Internal Conflict: validation accuracy = 92.75% (trained on 500 events, validation on 110)


Relevant Event: validation accuracy = 68%, 50%
Balanced accuracy: 50.00%

balanced accuracy: = 0.4999999456521798


In [ ]:
def cross_validation(kernel, C, gamma, num_features, folds):
    kf = KFold(n_splits=folds, shuffle=True)
    accuracy = []
    SVM = svm.SVC(C=C, kernel=kernel, degree=3, gamma=gamma)
    vectorizer_tf = TfidfVectorizer(max_features=num_features)
    encoder = LabelEncoder()
    X = vectorizer_tf.fit_transform(cleaned_events['description']).toarray()
    X_scaled = preprocessing.scale(X)
    Y = encoder.fit_transform(cleaned_events['relevant_event'])
    for train_idx, valid_idx in kf.split(X):
        split_X_train, split_X_valid = X_scaled[train_idx, :], X_scaled[valid_idx, :]
        split_Y_train, split_Y_valid = Y[train_idx], Y[valid_idx]
        SVM.fit(split_X_train,split_Y_train)
        predictions_SVM = SVM.predict(split_X_valid)
        accuracy.append(accuracy_score(predictions_SVM, split_Y_valid))
    return np.mean(accuracy)

    